In [ ]:
! pip install --upgrade pip
! pip install tensorflow-gpu
! pip install --upgrade neural_structured_learning

In [1]:
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, Normalizer
import pickle
from sklearn.model_selection import train_test_split

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, LeakyReLU
from tensorflow.keras.optimizers import Adam, SGD
import neural_structured_learning as nsl # 剛發布的套件!!

In [4]:
train = pd.read_csv('train.csv', index_col=0)
test = pd.read_csv('test.csv', index_col=0)
train.shape, test.shape

((14364, 55), (756, 54))

In [5]:
# 處理y
y_train = train['Cover_Type']-1 #減一是為了把 {1,...,7}變更範圍到{0,...,6}
y_train = to_categorical(y_train)
del train['Cover_Type']

In [6]:
def data_prepro(data, s=None):  
    if s is None:
        s = StandardScaler()
        s.fit(data)
        pickle.dump( s, open('s.pkl', 'wb') )

    data = pd.DataFrame( s.transform(data), columns=data.columns, index=data.index)
    return data

# train和test一起做preprocess
X = np.concatenate( (train, test), 0 )
X = pd.DataFrame( X , columns=train.columns, index=list(train.index)+list(test.index) )
X = data_prepro(X)
X.shape

(15120, 54)

In [7]:
x_train = X.values[:train.shape[0], :]
x_test = X.values[train.shape[0]:, :]
x_train.shape

(14364, 54)

In [8]:
model = Sequential()

In [9]:

model.add(Dense(units=400,                    #建立輸入層至隱藏層連接
           input_dim=54,                      #輸入神經元數目=784
           kernel_initializer='he_normal',  #以常態分佈亂數初始化參數
           activation=LeakyReLU(0.1))) 
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(units=200,                    #建立輸入層至隱藏層連接
           input_dim=400,                      #輸入神經元數目=784
           kernel_initializer='he_normal',  #以常態分佈亂數初始化參數
           activation=LeakyReLU(0.1))) 
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(units=100,                    #建立輸入層至隱藏層連接
           input_dim=200,                      #輸入神經元數目=784
           kernel_initializer='he_normal',  #以常態分佈亂數初始化參數
           activation=LeakyReLU(0.1))) 
model.add(Dense(units=50,                    #建立輸入層至隱藏層連接
           input_dim=100,                      #輸入神經元數目=784
           kernel_initializer='he_normal',  #以常態分佈亂數初始化參數
           activation=LeakyReLU(0.1))) 
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(units=20,                    #建立輸入層至隱藏層連接
           input_dim=50,                      #輸入神經元數目=784
           kernel_initializer='he_normal',  #以常態分佈亂數初始化參數
           activation=LeakyReLU(0.1))) 
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(units=7,                    #建立輸入層至隱藏層連接
           input_dim=20,                      #輸入神經元數目=784
           kernel_initializer='normal',  #以常態分佈亂數初始化參數
           activation='softmax')) 

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 400)               22000     
_________________________________________________________________
dropout (Dropout)            (None, 400)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 400)               1600      
_________________________________________________________________
dense_1 (Dense)              (None, 200)               80200     
_________________________________________________________________
dropout_1 (Dropout)          (None, 200)               0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 200)               800       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               2

In [11]:
# Wrap the model with adversarial regularization.
adv_config = nsl.configs.make_adv_reg_config(multiplier=0.2, adv_step_size=0.05)
adv_model = nsl.keras.AdversarialRegularization(model, adv_config=adv_config)

In [12]:
adv_model.compile(loss='categorical_crossentropy',   
            optimizer=Adam(lr=3e-3),                     
            metrics=['accuracy']) 
train_history=adv_model.fit({'feature': x_train, 'label': y_train},     
                        epochs=50,                      
                        batch_size=256) 


Epoch 1/50
14364/14364 [==============================] - 1s 60us/sample - loss: 1.3096 - categorical_crossentropy: 1.3022 - categorical_accuracy: 0.5506
Epoch 2/50
14364/14364 [==============================] - 0s 30us/sample - loss: 0.8703 - categorical_crossentropy: 0.8684 - categorical_accuracy: 0.6579
Epoch 3/50
14364/14364 [==============================] - 0s 29us/sample - loss: 0.7801 - categorical_crossentropy: 0.7781 - categorical_accuracy: 0.6866
Epoch 4/50
14364/14364 [==============================] - 0s 29us/sample - loss: 0.7175 - categorical_crossentropy: 0.7199 - categorical_accuracy: 0.7146
Epoch 5/50
14364/14364 [==============================] - 0s 29us/sample - loss: 0.6867 - categorical_crossentropy: 0.6897 - categorical_accuracy: 0.7240
Epoch 6/50
14364/14364 [==============================] - 0s 30us/sample - loss: 0.6741 - categorical_crossentropy: 0.6811 - categorical_accuracy: 0.7267
Epoch 7/50
14364/14364 [==============================] - 0s 29us/sample - 

In [16]:
adv_model.compile(loss='categorical_crossentropy',   
            optimizer=Adam(lr=3e-4),                     
            metrics=['accuracy'])
train_history=adv_model.fit({'feature': x_train, 'label': y_train},                
                        epochs=50,                      
                        batch_size=256, )

Epoch 1/50
14364/14364 [==============================] - 1s 61us/sample - loss: 0.3736 - categorical_crossentropy: 0.3733 - categorical_accuracy: 0.8548
Epoch 2/50
14364/14364 [==============================] - 0s 30us/sample - loss: 0.3603 - categorical_crossentropy: 0.3652 - categorical_accuracy: 0.8568
Epoch 3/50
14364/14364 [==============================] - 0s 30us/sample - loss: 0.3564 - categorical_crossentropy: 0.3558 - categorical_accuracy: 0.8606
Epoch 4/50
14364/14364 [==============================] - 0s 29us/sample - loss: 0.3550 - categorical_crossentropy: 0.3613 - categorical_accuracy: 0.8626
Epoch 5/50
14364/14364 [==============================] - 0s 29us/sample - loss: 0.3507 - categorical_crossentropy: 0.3515 - categorical_accuracy: 0.8606
Epoch 6/50
14364/14364 [==============================] - 0s 29us/sample - loss: 0.3536 - categorical_crossentropy: 0.3537 - categorical_accuracy: 0.8620
Epoch 7/50
14364/14364 [==============================] - 0s 30us/sample - l

In [17]:
adv_model.compile(loss='categorical_crossentropy',   
            optimizer=Adam(lr=1e-5),                     
            metrics=['accuracy'])
train_history=adv_model.fit({'feature': x_train, 'label': y_train},                  
                        epochs=50,                      
                        batch_size=256, )

Epoch 1/50
14364/14364 [==============================] - 1s 60us/sample - loss: 0.3087 - categorical_crossentropy: 0.3107 - categorical_accuracy: 0.8792
Epoch 2/50
14364/14364 [==============================] - 0s 29us/sample - loss: 0.3097 - categorical_crossentropy: 0.3088 - categorical_accuracy: 0.8766
Epoch 3/50
14364/14364 [==============================] - 0s 30us/sample - loss: 0.3079 - categorical_crossentropy: 0.3137 - categorical_accuracy: 0.8776
Epoch 4/50
14364/14364 [==============================] - 0s 29us/sample - loss: 0.3010 - categorical_crossentropy: 0.3036 - categorical_accuracy: 0.8826
Epoch 5/50
14364/14364 [==============================] - 0s 30us/sample - loss: 0.3101 - categorical_crossentropy: 0.3203 - categorical_accuracy: 0.8777
Epoch 6/50
14364/14364 [==============================] - 0s 29us/sample - loss: 0.3116 - categorical_crossentropy: 0.3125 - categorical_accuracy: 0.8774
Epoch 7/50
14364/14364 [==============================] - 0s 30us/sample - l

In [18]:
model.save_weights('model_20190917_NSL_1.h5')

In [30]:
# train dataset result
pred = np.argmax( model.predict(x_train), 1)+1
y_true = np.argmax( y_train, 1)+1
np.sum(pred==y_true)/len(x_train)

0.9161097187412977

In [31]:
pred = np.argmax( model.predict(x_test), 1)+1
df = pd.DataFrame( {'id': test.index, 'class':pred}, index=range(len(x_test)))
df.to_csv('sub_201909_NSL.csv', index=False)